<a href="https://colab.research.google.com/github/codylw2/CourseProject/blob/main/colab_cranfield_metapy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Authenticate for Google Drive

There is a file size limitation for my github repository so I have added the files that will be used for this session into my Google Drive and made them publicly shareable. In order to access them you must authenticate to Google. It should not matter what account you use to do this since the files are public.

In [ ]:
!pip install -q PyDrive
!pip install -q metapy
!pip install -q pytoml

In [20]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Acquire Files

In [ ]:
!git clone https://github.com/codylw2/CourseProject.git

In [ ]:
%cd /content/CourseProject/competition
!mkdir json_data
%cd json_data

In [24]:
# test_docs.json https://drive.google.com/file/d/1XqHy17_eOGk-BE91AC3gmKyJ1PiEjc6A/view?usp=sharing
downloaded = drive.CreateFile({'id':"1XqHy17_eOGk-BE91AC3gmKyJ1PiEjc6A"})
downloaded.GetContentFile('test_queries.json')

# train_docs.json : https://drive.google.com/file/d/1iyJ5F1BAT6BFKLOyumMt6z8jUaJ6KZLQ/view?usp=sharing
downloaded = drive.CreateFile({'id':"1XrBInztxbKW9FdNn8Tso9wPvnzkLbGaT"})
downloaded.GetContentFile('test_docs.json')

# train_queries.json : https://drive.google.com/file/d/1Dp2ExBJtUBE3UOpSuh1AwQnCH6vbYnaD/view?usp=sharing
downloaded = drive.CreateFile({'id':"1Dp2ExBJtUBE3UOpSuh1AwQnCH6vbYnaD"})
downloaded.GetContentFile('train_queries.json')

# train_qrels.json : https://drive.google.com/file/d/1tyGyuYtbGJHcKQIYoF4yaYCL9HRF2hku/view?usp=sharing
downloaded = drive.CreateFile({'id':"1tyGyuYtbGJHcKQIYoF4yaYCL9HRF2hku"})
downloaded.GetContentFile('train_qrels.json')

# test_docs.json https://drive.google.com/file/d/1XrBInztxbKW9FdNn8Tso9wPvnzkLbGaT/view?usp=sharing
downloaded = drive.CreateFile({'id':"1iyJ5F1BAT6BFKLOyumMt6z8jUaJ6KZLQ"})
downloaded.GetContentFile('train_docs.json')


# Create Cranfield Datasets

In [ ]:
%cd /content/CourseProject/competition/cranfield_metapy

WORKDIR = !pwd
WORKDIR = WORKDIR[0]
BASE = WORKDIR + "/../.."
DATASET_DIR = BASE + "/competition/datasets"
JSON_DIR = BASE + "/competition/json_data"


In [ ]:
!python $WORKDIR/create_cranfield.py \
    --run_type "train;test" \
    --query_keys "query" \
    --doc_keys "title:abstract:intro" \
    --cranfield_dir $WORKDIR \
    --input_dir $JSON_DIR

# Generate Predictions

In [ ]:
!python $WORKDIR/search_eval.py \
    --run_type "test" \
    --ranker "bm25+" \
    --params "1.0;0.38;1.0" \
    --dat_keys "title" \
    --doc_weights "1.0" \
    --cranfield_dir $WORKDIR \
    --predict_dir $BASE \
    --remove_idx


In [ ]:
!cat $BASE/predictions.txt